### CREAMOS EL ARCHIVO SQL


In [0]:
with open("consulta.sql", "w") as file:
    file.write("SELECT * FROM Ejemplo;")

#PROBAMOS QUE FUNCIONA BIEN

In [0]:
import os
import pandas as pd
from Lector_deConsulta import ConsultaDB

# 1) Instanciar y ejecutar consulta
lector = ConsultaDB()            # lee config.json y prepara conexión
df = lector.ejecutar_consulta()  # ejecuta consulta.sql

# 2) Exportar a Parquet
lector.exportar(df, nombre_archivo="mi_parquet", formato="parquet")

# 3) Leer el Parquet recién generado
ruta_parquet = os.path.join(lector.output_dir, "mi_parquet.parquet")
df_parquet = pd.read_parquet(ruta_parquet)

# 4) Imprimir y mostrar
print("DataFrame desde .parquet:")
display(df_parquet)


✅ Archivo guardado en: /Workspace/Users/azzure_roma_29@hotmail.com/ENTREGAS/Carpeta_ejemplo/mi_parquet.parquet
DataFrame desde .parquet:


Dinero,Valor1,Valor2,Valo3,Valor4
5,3,7,1,9
2,6,8,4,3
10,1,5,7,2
6,9,3,8,1
4,7,2,6,10
9,3,4,2,7
7,2,10,3,6
1,8,9,5,4
3,5,6,10,8
8,4,1,9,5


## FUNCIONA PERO TOCA CAMBIAR EL DF POR UNO DE MAYOR TAMAÑO 

In [0]:
# 0. Importaciones necesarias
from Lector_deConsulta import ConsultaDB
import pandas as pd
import shap
from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score

# 1. Instanciar la clase
lector = ConsultaDB()

# 2. Obtener los datos
df = lector.ejecutar_consulta()

# O un archivo plano:
# df = lector.cargar_archivo("/dbfs/FileStore/datos/mi_archivo.csv")

# 3. Limpieza inicial
df = df.drop_duplicates()
df = df.loc[:, ~(df.isna().all() | (df.nunique() == 1))]

# 4. Separar en variables explicativas y objetivo
objetivo = "Dinero"
X = df.drop(columns=[objetivo])
y = df[objetivo]

# 5. Separar entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 6. Preprocesamiento: separar numéricas y categóricas
num_cols = X.select_dtypes(include=["number"]).columns.tolist()
cat_cols = [col for col in X.columns if col not in num_cols]

preprocesador = ColumnTransformer(
    transformers=[
        ("num", "passthrough", num_cols),
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols),
    ]
)
#supongo que el dataFrame es grande (va a dar error dado que el nuestro es pequeño)
modelo_base = LGBMRegressor(n_estimators=300, learning_rate=0.05, random_state=42)
pipeline = Pipeline(steps=[("prep", preprocesador), ("model", modelo_base)])
pipeline.fit(X_train, y_train)

# 7. Importancia de variables con SHAP
explainer = shap.Explainer(pipeline["model"])
valores_shap = explainer(pipeline["prep"].transform(X_train))

importancia = pd.DataFrame({
    "variable": X_train.columns,
    "importancia": valores_shap.abs.mean(0).values
}).sort_values("importancia", ascending=False)

display(importancia)

# 8. Selección automática de variables (hasta cubrir 95 % de importancia)
cumsum = importancia["importancia"].cumsum() / importancia["importancia"].sum()
seleccionadas = importancia.loc[cumsum <= 0.95, "variable"].tolist()

print(f"Variables seleccionadas ({len(seleccionadas)}):", seleccionadas)

# 9. Crear subconjuntos reducidos
X_train_red = X_train[seleccionadas]
X_test_red = X_test[seleccionadas]

# 10. Modelo final con variables seleccionadas
pipeline_final = Pipeline(steps=[
    ("prep", ColumnTransformer(
        transformers=[
            ("num", "passthrough", [c for c in seleccionadas if c in num_cols]),
            ("cat", OneHotEncoder(handle_unknown="ignore"), [c for c in seleccionadas if c in cat_cols]),
        ])),
    ("model", LGBMRegressor(n_estimators=300, learning_rate=0.03, random_state=42)),
])

pipeline_final.fit(X_train_red, y_train)
predicciones = pipeline_final.predict(X_test_red)

print("R² en prueba:", r2_score(y_test, predicciones).round(4))

# 11. Exportar resultados
lector.exportar(df, nombre_archivo="datos_completos", formato="parquet")
lector.exportar(importancia, nombre_archivo="importancia_variables", formato="csv")


[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 24, number of used features: 0
[LightGBM] [Info] Start training from score 5.625000
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the spli

Uploading artifacts:   0%|          | 0/9 [00:00<?, ?it/s]

variable,importancia
Valor1,0.0
Valor2,0.0
Valo3,0.0
Valor4,0.0


Variables seleccionadas (0): []


---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
File <command-5255411454592102>, line 79
     69 # 10. Modelo final con variables seleccionadas
     70 pipeline_final = Pipeline(steps=[
     71     ("prep", ColumnTransformer(
     72         transformers=[
   (...)
     76     ("model", LGBMRegressor(n_estimators=300, learning_rate=0.03, random_state=42)),
     77 ])
---> 79 pipeline_final.fit(X_train_red, y_train)
     80 predicciones = pipeline_final.predict(X_test_red)
     82 print("R² en prueba:", r2_score(y_test, predicciones).round(4))

File /databricks/python/lib/python3.11/site-packages/mlflow/utils/autologging_utils/safety.py:578, in safe_patch.<locals>.safe_patch_function(*args, **kwargs)
    576     patch_function.call(call_original, *args, **kwargs)
    577 else:
--> 578     patch_function(call_original, *args, **kwargs)
    580 session.state = "succeeded"
    582 try_lo

# INSTALAMOS LAS LIBRERIAS NECESARIAS

In [0]:
%pip install -r "/Workspace/Users/azzure_roma_29@hotmail.com/ENTREGAS/requirements.txt"


  Obtaining dependency information for lightgbm from https://files.pythonhosted.org/packages/42/86/dabda8fbcb1b00bcfb0003c3776e8ade1aa7b413dff0a2c08f457dace22f/lightgbm-4.6.0-py3-none-manylinux_2_28_x86_64.whl.metadata
  Obtaining dependency information for shap from https://files.pythonhosted.org/packages/c8/ae/4a16ad24420966a6e3b71aa359756ab3314da38a0bc7e5ca83058814c9a9/shap-0.47.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for tqdm>=4.27.0 from https://files.pythonhosted.org/packages/d0/30/dc54f88dd4a2b5dc8a0279bdd7270e735851848b762aeb1c1184ed1f6b14/tqdm-4.67.1-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/57.7 kB ? eta -:--:--
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 5.0 MB/s eta 0:00:00
  Obtaining dependency information for slicer==0.0.8 from https://files.pythonhosted.org/packages/63/81/9ef641ff4e12cbcca30e54e72fb0951a2ba195d0cda0ba4100e532d929db/slicer-0.0.8-py3-none-any.

In [0]:
dbutils.library.restartPython()

